In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use("ggplot")
import pandas as pd
import random as rand
import string
import math
chars = string.ascii_uppercase + string.digits
%matplotlib inline
plt.rcParams["figure.figsize"] = (16,8)
import seaborn as sns
import sklearn as skl
import skimage as ski
from sklearn.preprocessing import LabelEncoder
from scipy import spatial
from sklearn.utils import shuffle
from os import system
import skimage as ski
from skimage.color import rgb2gray
from scipy import misc
from skimage.exposure import adjust_gamma

In [6]:
NMICROSATS = 5
NALLELEPERMICROSAT = 3
POPSIZE = 100
DISPERSAL_DST = .5 #use everything in KM
topology = ski.img_as_float(rgb2gray(misc.imread("roads/noroad.png")))
topology.shape
population = pd.DataFrame(columns = ['Sex', 'Age', 'Position'] + ['mSat'+ str(i+1).zfill(2) for i in range(NMICROSATS)])
population.index.name = 'name'

In [7]:
def generate_random_coyote(N):
    df = pd.DataFrame( columns = ['Sex', 'Age'])
    df.index.name='name'
    for i in range(N):
        new_coyote = pd.DataFrame()
        new_coyote['name'] = ["Coyote"+"".join([rand.choice(chars) for k in range(6)])]
        new_coyote["Position"] = [np.array([rand.uniform(0,topology.shape[0]-1),rand.uniform(0,topology.shape[1]-1)])]
        new_coyote['Sex'] = np.random.choice(["male", "female"], 1)
        new_coyote['Age'] =  np.random.normal(3, 1., 1)
        for n in range(NMICROSATS):
            new_coyote["mSat"+ str(n+1).zfill(2)] = [tuple([rand.choice(string.ascii_uppercase[:NALLELEPERMICROSAT]), rand.choice(string.ascii_uppercase[:NALLELEPERMICROSAT])])]
        
        new_coyote = new_coyote.set_index(new_coyote.name)
        del new_coyote["name"]
        
        df =df.append(new_coyote)
    return df
        

    
def coyote_puppy(momma, poppa, N):
    global population
    df = pd.DataFrame( columns = ['Boldness', 'Sex', 'Age'])
    df.index.name='name'
    for i in range(N):
        puppy = pd.DataFrame()
        puppy['name'] = ["Coyote"+"".join([rand.choice(chars) for k in range(6)])]
        
        puppy['Age'] =[.5]
        puppy["Position"] = [np.array([momma.Position[0]+rand.uniform(-3,3), momma.Position[1]+rand.uniform(-3,3)])]
        puppy['Sex'] = np.random.choice(["male", "female"], 1)
        for n in range(NMICROSATS):
            alleles_from_parents = [np.random.choice(momma["mSat"+ str(n+1).zfill(2)], 1)[0], np.random.choice(poppa["mSat"+ str(n+1).zfill(2)], 1)[0]]
            if rand.random() < 10**(-3): #mutation rate
                print("new allele")
                which_allele_to_mutate = rand.randint(0,1) 
                alleles_from_parents[which_allele_to_mutate] += rand.choice(string.ascii_lowercase)
            puppy["mSat"+ str(n+1).zfill(2)] = [tuple(alleles_from_parents)]
        puppy = puppy.set_index(puppy.name)
        del puppy["name"]
        df =df.append(puppy)
    
    population = pd.concat([population,df])
    


In [10]:
population = pd.concat([population, generate_random_coyote(POPSIZE)])

population.head()


,Age,Position,Sex,mSat01,mSat02,mSat03,mSat04,mSat05
name,,,,,,,,
CoyoteKFBLK4,3.974987,"[27.370347693, 589.819329364]",female,"(C, A)","(C, B)","(B, B)","(A, B)","(B, A)"
CoyoteZ6I2VQ,4.661364,"[485.201774409, 936.709844647]",female,"(B, C)","(C, A)","(B, C)","(C, A)","(A, A)"
CoyoteZ0NU4H,1.452434,"[20.5922519562, 332.464114311]",male,"(A, B)","(A, A)","(C, A)","(C, A)","(A, C)"
CoyoteLG89CW,2.768173,"[399.700125344, 706.040330853]",female,"(A, A)","(A, C)","(A, B)","(A, C)","(C, C)"
CoyoteFJX1RU,1.998592,"[450.779063305, 16.7633311789]",female,"(B, A)","(A, A)","(C, B)","(A, C)","(C, A)"


In [ ]:
def disperse(y,x):
    global topology
    angle = 2*math.pi*rand.random()
    distance = np.array([math.sin(angle), math.cos(angle)])
    pfy, pfx = position[0]+distance[0], position[1]+distance[1]
    
    
    if pfx > (topology.shape[1]-1): pfx = 2*(topology.shape[1]-1) - (distance[1] + position[1]) 
    if pfy > (topology.shape[0]-1): pfy = 2*(topology.shape[0]-1) - (distance[0] + position[0]) 
    if pfx < 0: pfx = abs(pfx)
    if pfy < 0: pfy = abs(pfy)
    